In [1]:
import torch
from torch.nn.utils import clip_grad_value_
# from transformers.models.gemma3.modeling_gemma3 import Gemma3TextScaledWordEmbedding
from functions import *
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-pt-unsloth-bnb-4bit",
    max_seq_length = 8192, # Choose any for long context!
    load_in_4bit = True,
    resize_model_vocab=24,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-29 20:29:27 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.4.3: Fast Gemma3 patching. Transformers: 4.51.3. vLLM: 0.8.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.642 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [2]:
model.config.image_token_index = 16 # new image token
model.language_model.lm_head.weight.requires_grad_(True);
# # model.language_model.lm_head.load_state_dict(torch.load("/home/zhenlan/Desktop/Projects/ARC2/Model/gemma24.pth"))
model.language_model.lm_head.load_state_dict(torch.load("../../Model/lm_heads_weights_VLM2.pth"))
model = PeftModel.from_pretrained(model, "../../Model/merged_model_VLM2", is_trainable=False)

In [3]:
model.eval()
model.language_model.model.gradient_checkpointing = False

In [4]:
import json
output_path = '/home/zhenlan/Desktop/Projects/ARC2/Data/ARC-AGI-2-main/combined_data.json'
with open(output_path, 'r') as f:
    data = json.load(f)

In [5]:
inputs_f, y_f = next(iter(data_gen_VLM(data, False, tokenizer, 3, False)))
inputs_t, y_t = next(iter(data_gen_VLM(data, False, tokenizer, 3, True)))
start_idx = len(inputs_t['input_ids'][0]) - 1
end_idx = len(inputs_f['input_ids'][0]) - 2

In [6]:
start_idx, end_idx

(3887, 4756)

In [7]:
# logit1 and logit2 give different results, suggesting something is sequence length dependent.
# tested mannual casual mask, still see the difference. So not caused by mask.
# model.eval();
# with torch.no_grad():
#     with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
#         outputs_full = model(**inputs_f)
#         logits1 = outputs_full.logits[0, :start_idx+1]

# # shorten inputs
# inputs_f['input_ids'] = inputs_f['input_ids'][:, :start_idx+1]
# # inputs_f['attention_mask'] = inputs_f['attention_mask'][:, :, :start_idx+1, :start_idx+1]
# inputs_f['attention_mask'] = inputs_f['attention_mask'][:, :start_idx+1]
# inputs_f['token_type_ids'] = inputs_f['token_type_ids'][:, :start_idx+1]

# with torch.no_grad():
#     with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
#         outputs_full = model(**inputs_f)
#         logits2 = outputs_full.logits
# torch.sum(logits1.argmax(-1) == logits2.argmax(-1))/logits1.shape[0]
# torch.mean(torch.abs(logits1 - logits2)) # original mask, no peft
# torch.mean(torch.abs(logits1 - logits2)) # casual mask
# torch.mean(torch.abs(logits1 - logits2)) # original mask

In [8]:
model.eval();
with torch.no_grad():
    with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
        outputs_full = model(**inputs_f)
        logits1 = outputs_full.logits[0, start_idx: end_idx]

In [9]:
logits2 = []

with torch.no_grad():
    # Initial pass with the input prompt
    with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
        outputs_kv_init = model(**inputs_t, use_cache=True)
        logits2.append(outputs_kv_init.logits[0, -1, :])
        # Store the KV cache from initial pass
        past_key_values = outputs_kv_init.past_key_values

    # Iterate through the continuation tokens
    for i in range(end_idx - start_idx - 1):
        # Prepare input for the next iteration: only the current continuation token
        current_token_id_tensor = inputs_f['input_ids'][:, start_idx + i + 1].unsqueeze(1)

        # Model call with the single next token and past_key_values
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            outputs_kv_step = model(input_ids=current_token_id_tensor, 
                                   past_key_values=past_key_values, 
                                   use_cache=True)
            # Store the logits from this step - CORRECTION HERE
            logits2.append(outputs_kv_step.logits[0, -1, :])
            # Update KV cache with new values
            past_key_values = outputs_kv_step.past_key_values

In [11]:
logits2 = torch.stack(logits2, dim=0)

In [12]:
torch.mean(torch.abs(logits1 - logits2),-1)

tensor([0.1914, 1.7734, 1.4766, 1.5547, 1.8594, 2.0000, 1.8281, 2.0781, 2.0312,
        2.1094, 2.1875, 2.2031, 2.3281, 2.2656, 2.1094, 2.0312, 1.8906, 1.9922,
        2.0781, 2.2031, 2.0312, 1.7500, 1.7188, 1.9062, 2.0312, 2.0156, 2.0625,
        1.9922, 2.0938, 2.2500, 1.1953, 1.6641, 1.9219, 1.6797, 1.9297, 2.4219,
        2.5781, 2.3906, 3.2812, 2.8594, 1.9766, 2.3750, 2.5312, 2.5781, 1.9766,
        2.5938, 2.0469, 2.0312, 2.5156, 2.4062, 2.3594, 2.0312, 2.2656, 1.8750,
        2.0781, 2.3281, 2.2188, 2.1562, 2.3281, 3.3125, 2.4844, 1.8438, 2.1562,
        2.2969, 2.5156, 2.6719, 1.9062, 2.2812, 2.2344, 2.4219, 2.5625, 2.5469,
        2.7656, 2.0469, 2.5156, 2.0000, 1.9062, 2.0625, 1.8203, 1.8594, 2.1094,
        2.1719, 1.5469, 1.6406, 1.8125, 2.3125, 2.4531, 1.4609, 2.2031, 3.9688,
        2.7031, 2.3281, 1.9844, 1.8438, 2.3438, 2.0781, 2.1094, 2.4531, 2.2344,
        2.2031, 3.0000, 2.5000, 2.9219, 2.6875, 2.8438, 2.2969, 1.5938, 1.5391,
        1.8750, 2.1406, 1.4219, 2.2031, 